In [1]:
from transformers import BertModel, BertTokenizer

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [4]:
model = BertForSequenceClassification.from_pretrained("my_bert_model")

In [44]:
# Example usage
sample_response = "stacks are implemented using arrays and linked list "
student_response = "queues"

In [45]:
# Ensure the sample_response and student_response are strings
sample_response = str(sample_response)
student_response = str(student_response)

In [46]:
label_mapping = {0: 0, 0.5: 1, 1: 2, 1.5: 3, 2: 4, 2.5: 5}

In [47]:
def predict_grade(sample_response, student_response, model, tokenizer):
    #model.eval()

    # Convert to strings
    sample_response = str(sample_response)
    student_response = str(student_response)

    # Tokenize and encode the input
    encoding = tokenizer(
        sample_response,
        student_response,
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='pt'
    )

    # Move tensors to device
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Make the prediction
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()

    # Map the predicted label back to the original grade
    grade_mapping_reverse = {v: k for k, v in label_mapping.items()}
    predicted_grade = grade_mapping_reverse[predicted_label]

    return predicted_grade, predicted_label


In [48]:
import torch

In [49]:
# Make the prediction
predicted_grade, predicted_label = predict_grade(sample_response, student_response, model, tokenizer)

print(f"Predicted Grade: {predicted_grade}")
print(f"Predicted Label: {predicted_label}")

Predicted Grade: 2
Predicted Label: 4


In [50]:
model_name = 'bert-base-uncased'
model1 = BertModel.from_pretrained(model_name)
tokenizer1 = BertTokenizer.from_pretrained(model_name)

In [51]:
def get_bert_embeddings(text, model, tokenizer):
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Get BERT model embeddings
    with torch.no_grad():
        outputs = model(input_ids)
    
    # Extract the embeddings for the [CLS] token
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    
    return embeddings

In [52]:
def calculate_cosine_similarity(embedding1, embedding2):
    # Reshape to (1, -1) to match the shape expected by cosine_similarity
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    
    return similarity[0][0]

In [53]:
# Get BERT embeddings for the texts
embedding1 = get_bert_embeddings(sample_response, model1, tokenizer)
embedding2 = get_bert_embeddings(student_response, model1, tokenizer)

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
 similarity_score = calculate_cosine_similarity(embedding1, embedding2)

print(f"Cosine Similarity Score: {similarity_score}")

Cosine Similarity Score: 0.8669043183326721


In [56]:
print(f"Score: {similarity_score*predicted_grade}")

Cosine Similarity Score: 1.7338086366653442
